TEAM MEMBERS:-

Priyanka Boral(MT19127)


Reecha Kumari Giri(MT19134)


Akanksha Dewangan(MT19049)



In [ ]:
#files are loaded in data frame.
import pandas as pd
df_train = pd.read_csv('train_final_amino_acid_result.csv')
df_test = pd.read_csv('valid_final_amino_acid_result.csv')

#df_train_dipep = pd.read_csv('final_dipeptide_result_train.csv')
#df_test_dipep = pd.read_csv('final_dipeptide_result_valid.csv')

In [ ]:
#test labels
import pandas as pd
#merging of dipeptide and amino acid 
#df_train = pd.read_csv('merged_train.csv')
#df_test = pd.read_csv('merged_test.csv')

del df_train['ID']
#test labels 
test_labels=df_test['ID']
del df_test['ID']


In [ ]:
#train data
df_train

,AAC_A,AAC_C,AAC_D,AAC_E,AAC_F,AAC_G,AAC_H,AAC_I,AAC_K,AAC_L,AAC_M,AAC_N,AAC_P,AAC_Q,AAC_R,AAC_S,AAC_T,AAC_V,AAC_W,AAC_Y,Type
0,9.42,0.00,7.97,9.42,0.72,7.25,2.17,5.07,4.35,17.39,4.35,2.90,5.07,2.17,2.90,6.52,4.35,7.25,0.00,0.72,DNA
1,17.46,1.59,0.00,7.94,0.00,1.59,6.35,3.17,6.35,7.94,1.59,4.76,3.17,7.94,9.52,7.94,4.76,4.76,1.59,1.59,DNA
2,6.90,6.90,6.90,3.45,3.45,3.45,6.90,6.90,0.00,3.45,0.00,0.00,6.90,0.00,20.69,10.34,6.90,3.45,0.00,3.45,DNA
3,5.43,2.17,2.72,6.52,2.72,8.70,3.26,4.35,8.15,9.24,2.72,2.17,7.61,5.98,7.07,8.15,3.80,7.07,0.00,2.17,DNA
4,6.17,4.94,2.47,3.09,2.47,11.73,6.79,3.70,2.47,8.64,0.00,6.17,8.02,4.94,4.32,3.09,6.79,8.64,3.09,2.47,DNA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,5.84,1.08,6.06,5.84,5.84,6.49,0.87,4.98,9.09,11.47,1.95,3.90,3.46,4.11,3.90,9.31,4.33,6.93,0.22,4.33,NDNA
3045,8.30,2.40,6.99,7.42,3.06,8.52,1.75,6.55,6.77,5.68,4.37,2.62,3.93,2.84,3.71,6.11,7.21,7.21,1.09,3.49,NDNA
3046,6.19,3.19,4.19,7.98,4.59,6.19,2.59,4.79,9.18,7.78,1.40,3.79,3.39,3.79,2.99,11.38,6.59,4.79,1.60,3.59,NDNA
3047,9.40,3.36,2.68,4.03,1.34,9.40,2.01,2.68,2.68,8.05,1.34,3.36,8.72,6.04,14.77,8.05,3.36,3.36,3.36,2.01,NDNA


In [ ]:
#label encoding to convert (1,-1) labels to (0,1) by preprocessing
from sklearn import preprocessing  
label_encoder = preprocessing.LabelEncoder() 
df_train['Type']= label_encoder.fit_transform(df_train['Type'])

In [ ]:
#label maping of classes {'DNA': 0, 'NDNA': 1}
label_mapping = dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_)))
label_mapping

{'DNA': 0, 'NDNA': 1}

In [ ]:
#X is train data
#y is target column
X = df_train.iloc[:,:-1]
y = df_train['Type']
# split test train data where test is 20% and train is 80%
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,shuffle=True)

In [ ]:
#Merged data
X1 = df_train1.iloc[:,:-1]
y1 = df_train1['Type']
from sklearn.model_selection import train_test_split
X_train1, X_valid1, y_train1, y_valid1 = train_test_split(X1, y1, test_size=0.2,shuffle=True)


SVM Model

In [ ]:
#SVM model over train data
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn import metrics
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)

svc = svm.SVC(kernel='linear', C=500, gamma=3).fit(X_train, y_train)
y_pred_svm = svc.predict(X_valid)

print('Accuracy on test set: ', metrics.accuracy_score(y_valid, y_pred_svm))

Accuracy on test set:  0.7065573770491803


In [ ]:
# Akku's output2.csv result is obtained on commenting X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2,shuffle=True) line
# SVM Model where cross validation of K-Fold is applied
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn import svm
scaler = StandardScaler()
X = scaler.fit_transform(X)
svc = svm.SVC(kernel='linear', C=500,gamma='auto').fit(X, y)
scores = cross_val_score(svc, X, y, cv=10)
# from sklearn import metrics
# print("Accuracy:",metrics.accuracy_score(y_valid, y_pred))

In [ ]:
#use this for svm for checking cross validation score
sum(scores)/len(scores)

0.7018647540983608

In [ ]:
#predicting for test data
svc = svm.SVC(kernel='linear', C=500, gamma=3).fit(X, y)
df_test=scaler.transform(df_test)
y_pred=svc.predict(df_test)

In [ ]:
y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [ ]:
# DNA=1, NDA=-1

In [ ]:
# label encoding again where 0 is encoded to 1 and 1 is encoded to -1.
y_pred_temp=[]
for i in y_pred:
  if i==0:
    y_pred_temp.append(1)
  else:
    y_pred_temp.append(-1)

In [ ]:
#copying to sample file
#conversion into output file with 2 column [ID,Labels]
data_res=pd.DataFrame()
data_res['ID']=test_labels
data_res['Lable']=y_pred_temp
data_res.to_csv (r'new_svm_output.csv', index = False, header=True)

Random Forest Model(Ensembling)

In [ ]:
# Random Forest along wiht k-fold 10 cross validation.
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import accuracy_score, roc_auc_score
#cross validation
kfold = model_selection.KFold(n_splits=10, random_state=5)
clf = RandomForestClassifier(n_estimators=135, random_state=None, n_jobs=4, criterion='entropy', max_features='auto')
clf.fit(X_train, y_train)
y_pred_rf = clf.predict(X_valid)
results = model_selection.cross_val_score(clf, X_train, y_train, cv=kfold)
print(results.mean())

#print('Accuracy on test set: ', accuracy_score(y_valid, y_pred_rf))

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.717523443297578


In [ ]:
#
y_pred=clf.predict(df_test)

In [ ]:
print(len(y_pred))

1071


In [ ]:
# label encoding again where 0 is encoded to 1 and 1 is encoded to -1.
y_pred_temp_rf=[]
for i in y_pred:
  if i==0:
    y_pred_temp_rf.append(1)
  else:
    y_pred_temp_rf.append(-1)

In [ ]:
#copying to sample file
data_res=pd.DataFrame()
data_res['ID']=test_labels
data_res['Lable']=y_pred_temp_rf
data_res.to_csv (r'output3_RF.csv', index = False,header=True)

MLP Classifier Model:

In [ ]:
#MLP
from sklearn.neural_network import MLPClassifier
clf_mlp=MLPClassifier(alpha=1e-05, hidden_layer_sizes=(150,120), random_state=1,
            solver='lbfgs')
clf_mlp.fit(X_train, y_train)
y_pred_mlp = clf_mlp.predict(X_valid)
print('Accuracy on valid set: ', accuracy_score(y_valid, y_pred_mlp))

Accuracy on valid set:  0.6983606557377049


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:470: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


Naive Bayes Classifier Model:

In [ ]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_valid)
print('Accuracy on test set: ', accuracy_score(y_valid, y_pred))

Accuracy on test set:  0.6672131147540984


AdaBoost Model:

In [ ]:
#Adaboost
import pandas
from sklearn import model_selection
from sklearn.ensemble import AdaBoostClassifier
seed = 7
num_trees = 30
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = model_selection.cross_val_score(model, X_train, y_train, cv=kfold)
print(results.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.6720130877690076


ExtraTree Classifier Model:

In [ ]:
import pandas
from sklearn import model_selection
from sklearn.ensemble import ExtraTreesClassifier

seed = 9
num_trees = 120
max_features = 8
kfold = model_selection.KFold(n_splits=10, random_state=seed)
model1 = ExtraTreesClassifier(n_estimators=num_trees, criterion='gini',min_samples_split=2, max_features=max_features, min_samples_leaf=1, min_weight_fraction_leaf=0, max_leaf_nodes=None, min_impurity_decrease=0.0, min_impurity_split=None, bootstrap=False, oob_score=False, n_jobs=-1, random_state=False, verbose=0, warm_start=False, class_weight=None, ccp_alpha=0, max_samples=None)
results = model_selection.cross_val_score(model1, X_train, y_train, cv=kfold)
print(results.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.7244940295486743


In [ ]:
model1.fit(X_train, y_train)
y_pred=model1.predict(df_test)

In [ ]:
y_pred_temp_rf=[]
for i in y_pred:
  if i==0:
    y_pred_temp_rf.append(1)
  else:
    y_pred_temp_rf.append(-1)

In [ ]:
#copying to sample file
data_res=pd.DataFrame()
data_res['ID']=test_labels
data_res['Lable']=y_pred_temp_rf
data_res.to_csv (r'output4_ETC.csv', index = False,header=True)

Voting Classifier Model:

In [ ]:
#Voting classifier
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
seed = 7
kfold = model_selection.KFold(n_splits=5, random_state=seed)
# create the sub models
estimators = []
#estimators.append(('mlp',clf_mlp))
#estimators.append(('svm',svc))
estimators.append(('randomforest',clf))
#estimators.append(('adaboost',model))
estimators.append(('Extratreeclassifier',model1))
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = model_selection.cross_val_score(ensemble, X, y, cv=kfold)
print(results.mean())

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:296: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


0.522752160219656


KNN Classifier:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=21)
neigh.fit(X, y)
kfold = model_selection.KFold(n_splits=10, random_state=42)
neigh.fit(X, y)
results = model_selection.cross_val_score(clf, X, y, cv=kfold)
print(results.mean())

XGBoost Classifier:

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
xgb = GradientBoostingClassifier( n_estimators=110, max_depth=70,max_features=110)
xgb.fit(X, y)
kfold = model_selection.KFold(n_splits=10, random_state=42)
xgb.fit(X, y)
results = model_selection.cross_val_score(clf, X, y, cv=kfold)
print(results.mean())

Voting classifier:

In [ ]:
eclf1 = VotingClassifier(estimators=[
      ('gnb', clf), ('ext', model),('rnd',rdf),('ada',model2)], voting='soft')

eclf1 = eclf1.fit(X, y)
results = model_selection.cross_val_score(eclf1, X, y, cv=kfold)
y_pred=eclf1.predict(df_test)
results.mean()

In [ ]:
y_pred_temp_rf=[]
for i in y_pred:
  if i==0:
    y_pred_temp_rf.append(1)
  else:
    y_pred_temp_rf.append(-1)

In [ ]:
#copying to sample file
data_res=pd.DataFrame()
data_res['ID']=test_labels
data_res['Lable']=y_pred_temp_rf
data_res.to_csv (r'output_Major_voting2.csv', index = False,header=True)